1. Importações e Inicializações

In [1]:
import json
import string
import random
import nltk
import numpy as np
import tkinter as tk
from tkinter import scrolledtext
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Inicializando o lemmatizer
lemmatizer = WordNetLemmatizer()


2. Definição das Funções do Chatbot

In [2]:
# Função para preprocessar os dados
def preprocess_data(data):
    words = []
    classes = []
    doc_X = []
    doc_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            tokens = nltk.word_tokenize(pattern)
            words.extend(tokens)
            doc_X.append(pattern)
            doc_y.append(intent["tag"])

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

    words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
    words = sorted(set(words))
    classes = sorted(set(classes))

    return words, classes, doc_X, doc_y

# Função para criar o saco de palavras e os rótulos
def create_training_data(words, classes, doc_X, doc_y):
    training = []
    out_empty = [0] * len(classes)

    for idx, doc in enumerate(doc_X):
        bow = [1 if word in doc.lower() else 0 for word in words]
        output_row = list(out_empty)
        output_row[classes.index(doc_y[idx])] = 1
        training.append([bow, output_row])

    random.shuffle(training)
    training = np.array(training, dtype=object)

    train_X = np.array(list(training[:, 0]))
    train_y = np.array(list(training[:, 1]))

    return train_X, train_y

# Função para construir o modelo de deep learning
def build_model(input_shape, output_shape, learning_rate=0.01, epochs=200):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(output_shape, activation="softmax"))

    adam = Adam(learning_rate=learning_rate, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
    return model

# Função para limpar o texto
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens

# Função para converter texto em saco de palavras
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [1 if word in tokens else 0 for word in vocab]
    return np.array(bow)

# Função para prever a classe
def pred_class(text, vocab, labels, model, threshold=0.2):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > threshold]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return [labels[r[0]] for r in y_pred]

# Função para obter resposta baseada na classe prevista
def get_response(intents_list, intents_json):
    tag = intents_list[0]
    for intent in intents_json["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])


3. Configuração dos Dados e Treinamento do Modelo

In [3]:
# Carregando e preprocessando os dados
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Oi", "Como vai você?", "E aí?", "Olá", "Você está bem?"],
              "responses": ["Olá!!", "Bem, e você?", "Tudo maravilha!", "Oi", "Muito melhor agora"],
             },
             {"tag": "age",
              "patterns": ["Você tem quantos anos?", "Quantos anos você tem?", "Quando é o seu aniversário?", "Quando você nasceu?"],
              "responses": ["Eu tenho 37 anos", "Eu nasci em 1983", "Meu aniversário é em dezembro", "05/12/1983"]
             },
             {"tag": "date",
              "patterns": ["Tem planos para o fim de semana?",
"O que você faz no tempo livre?", "Quais seus planos para este fim de semana?", "Vamos ao parque no fim de semana?"],
              "responses": ["Eu não tenho nada para fazer...", "Eu estou ocupado", "Pretendo dormir o tempo todo"]
             },
             {"tag": "name",
              "patterns": ["Qual o seu nome?", "Você tem apelido?", "Quem é você?", "Como você se chama?", "Quero saber seu nome"],
              "responses": ["Meu nome é Boop", "Sim, sou o Boop", "Boop."]
             },
             {"tag": "goodbye",
              "patterns": [ "Boa tarde", "Bom dia", "Boa noite", "tchau", "até mais", "obrigado", "adeus", "até"],
              "responses": ["Foi ótimo falar com você hoje", "Nos vemos depois", "Nos vemos em breve!"]
             }
]}

words, classes, doc_X, doc_y = preprocess_data(data)
train_X, train_y = create_training_data(words, classes, doc_X, doc_y)

# Construindo e treinando o modelo
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
model = build_model(input_shape, output_shape)
model.summary()
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)


c:\Users\lucas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\lucas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         6,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,365 (60.02 KB)

 Trainable params: 15,365 (60.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step - accuracy: 0.2308 - loss: 1.5580
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3462 - loss: 1.5389
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4615 - loss: 1.4470
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6538 - loss: 1.3718
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7692 - loss: 1.1918
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6923 - loss: 1.1283
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7308 - loss: 0.9443
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7692 - loss: 0.8717
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9231 - loss: 0.6579
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8462 - loss: 0.6343
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9231 - loss: 0.4987
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9615 - lo

4. Interface Gráfica com tkinter

In [4]:
# Função para enviar mensagens
def send_message():
    message = entry_box.get()
    chat_window.insert(tk.END, "Você: " + message + "\n")
    entry_box.delete(0, tk.END)

    intents = pred_class(message, words, classes, model)
    response = get_response(intents, data)
    chat_window.insert(tk.END, "Bot: " + response + "\n")

# Configuração da janela principal
root = tk.Tk()
root.title("Chatbot")
root.geometry("400x500")

# Janela de rolagem para exibir o chat
chat_window = scrolledtext.ScrolledText(root, bd=1, bg="white", width=50, height=8, font=("Arial", 14), wrap=tk.WORD)
chat_window.place(x=6, y=6, height=386, width=370)

# Caixa de entrada de texto
entry_box = tk.Entry(root, bd=0, bg="white", width=29, font=("Arial", 14))
entry_box.place(x=6, y=400, height=88, width=265)

# Botão para enviar mensagem
send_button = tk.Button(root, text="Enviar", width=12, height=5, bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command=send_message)
send_button.place(x=282, y=400, height=88)

root.mainloop()


5. Execução Completa do Chatbot

In [5]:
if __name__ == "__main__":
    # Carregar, processar dados e treinar modelo
    words, classes, doc_X, doc_y = preprocess_data(data)
    train_X, train_y = create_training_data(words, classes, doc_X, doc_y)
    input_shape = (len(train_X[0]),)
    output_shape = len(train_y[0])
    model = build_model(input_shape, output_shape)
    model.fit(x=train_X, y=train_y, epochs=200, verbose=1)
    
    # Iniciar a interface gráfica do chatbot
    root = tk.Tk()
    root.title("Chatbot")
    root.geometry("400x500")

    chat_window = scrolledtext.ScrolledText(root, bd=1, bg="white", width=50, height=8, font=("Arial", 14), wrap=tk.WORD)
    chat_window.place(x=6, y=6, height=386, width=370)

    entry_box = tk.Entry(root, bd=0, bg="white", width=29, font=("Arial", 14))
    entry_box.place(x=6, y=400, height=88, width=265)

    send_button = tk.Button(root, text="Enviar", width=12, height=5, bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command=send_message)
    send_button.place(x=282, y=400, height=88)

    root.mainloop()


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 642ms/step - accuracy: 0.1923 - loss: 1.6270
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2308 - loss: 1.5406
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5000 - loss: 1.3720
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6923 - loss: 1.2759
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6923 - loss: 1.1261
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4615 - loss: 1.1804
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6923 - loss: 0.9511
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8462 - loss: 0.7455
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9231 - loss: 0.5729
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8846 - loss: 0.6176
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8846 - loss: 0.5046
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9615 - l